# 피처 시각화 실습 (1)

## MNIST 데이터셋

### 데이터의 구성
* 이미지: 28x28 픽셀의 흑백 이미지로 70000개로 구성. 각 이미지는 손으로 쓴 하나의 숫자를 나타내고, 이미지는 0-255 사이의 값으로 각 픽셀의 그레이스케일 값을 가짐
* 레이블: 0부터 9까지의 숫자

In [ ]:
# [1]
from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

데이터 로드 & 전처리

In [ ]:
# [2]
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [ ]:
# [3]
num_classes = 2
img_rows, img_cols = 28, 28

# 데이터 선택
train_picks = np.logical_or(y_train==2,y_train==7)
test_picks = np.logical_or(y_test==2,y_test==7)

# 데이터 추출
x_train = x_train[train_picks]
x_test = x_test[test_picks]

y_train = np.array(y_train[train_picks]==7,dtype=int)
y_test = np.array(y_test[test_picks]==7,dtype=int)

# 이미지 형태 조정
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

# 이미지 정규화
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

# 레이블 인코딩
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# 데이터셋의 정보 출
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

CNN 모델 구성


In [ ]:
# [4]
# create model

model = Sequential()

model.add(Conv2D(32, [3, 3], input_shape = input_shape))
model.add(Activation("relu"))
model.add(Conv2D(64, [3, 3]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

hist = model.fit(x_train[:1000], y_train[:1000], batch_size=64, epochs=10, verbose=1, validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=1)
print()
print('Test score:', score[0])
print('Test accuracy:', score[1])

##1. Visualizing weights
###첫 번째 컨볼루션 레이어의 가중치 시각화


In [ ]:
# [5]
W1 = model.layers[0].get_weights()[0]           # 가중치 추출
W1 = model.layers[0].get_weights()[0][:,:,0,:]  # 가중치 배열 형태 조정

# 가중치 정보 확인
print(type(W1), W1.shape)
print()
print("Filter 1 in the first convolution layer : ")
print(W1[:,:,0])

plt.figure(1, figsize=(15,8))

# 필터 별 가중치 시각
for i in range(0,32):
    plt.subplot(4,10,i+1)
    plt.title('Filter ' + str(i+1))
    plt.imshow(W1[:,:,i],interpolation="nearest",cmap="gray")

plt.show()

###두 번째 컨볼루션 레이어의 가중치 시각화


In [ ]:
# [6]
W2 = model.layers[2].get_weights()[0]  # 가중치 추출
W2 = W2.reshape(9,32,64)               # 가중치 배열 형태 조정

# 가중치 정보 확인
print(type(W2), W2.shape)
print()
print("Filter 1 in the second convolution layer : ")
print(W2[:,:,0])

plt.figure(1, figsize=(15,8))

# 필터 별 가중치 시각
for i in range(0,64):
    plt.subplot(7,10,i+1)
    plt.title('Filter ' + str(i+1))
    plt.imshow(W2[:,:,i],interpolation="nearest",cmap="gray")

plt.show()

##2. 활성화 출력 시각화

###첫 번째 컨볼루션 레이어의 활성화 시각화

In [ ]:
# [7]
model2 = Sequential()
model2.add(Conv2D(32, [3, 3], weights=model.layers[0].get_weights(), input_shape=input_shape))
model2.add(Activation("relu"))

x_rep = model2.predict(x_test[0:10])
x_rep_1 = x_rep[0,:,:,:]  # only look into the first test sample, which is a 7 digit
print(x_rep_1.shape)

plt.figure(1, figsize=(15,8))

for i in range(0,32):
    plt.subplot(4,10,i+1)
    plt.title('Filter ' + str(i+1))
    plt.imshow(x_rep_1[:,:,i],interpolation="nearest",cmap="gray")

plt.show()

###두 번째 컨볼루션 레이어의 활성화 시각화

In [ ]:
# [8]
model3 = Sequential()
model3.add(Conv2D(32, [3, 3], weights=model.layers[0].get_weights(), input_shape=input_shape))
model3.add(Activation("relu"))
model3.add(Conv2D(64, [3, 3], weights=model.layers[2].get_weights()))
model3.add(Activation("relu"))

x_rep = model3.predict(x_test[0:10]) # representation of the first 10 test samples
x_rep_1 = x_rep[0,:,:,:]  # only look into the first test sample, which is a 7 digit
print(x_rep_1.shape)

plt.figure(1, figsize=(15,8))

for i in range(0,64):
    plt.subplot(7,10,i+1)
    plt.title('Filter ' + str(i+1))
    plt.imshow(x_rep_1[:,:,i],interpolation="nearest",cmap="gray")

plt.show()